In [1]:
import torch
print(f'PyTorch version: {torch.__version__}')
print('*'*10)
print(f'_CUDA version: ')
!nvcc --version
print('*'*10)
print(f'CUDNN version: {torch.backends.cudnn.version()}')
print(f'Available GPU devices: {torch.cuda.device_count()}')
print(f'Device Name: {torch.cuda.get_device_name()}')

PyTorch version: 2.0.1+cu118
**********
_CUDA version: 
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:41:10_Pacific_Daylight_Time_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
**********
CUDNN version: 8700
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
# Import the libraries
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder


In [3]:
# Creating loaders for data
data_dir = 'dtset' # Root dir
#Transformation for pictures
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to the input size required by AlexNet
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

#Torch folders for data 
train_dataset = ImageFolder(root=data_dir + '/train', transform=transform)
test_dataset = ImageFolder(root=data_dir + '/test', transform=transform)
val_dataset = ImageFolder(root=data_dir + '/val', transform=transform)

# Create data loaders
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [4]:
#Loading the model 
model = models.alexnet(pretrained=True)

c:\Users\erikm\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\erikm\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
num_classes = 7
model.classifier[6] = nn.Linear(4096, num_classes)

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001,weight_decay=0.001)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)


In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [24]:
# Train the model
num_epochs = 20
example_image = None
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        #scheduler.step()
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

Epoch 1/20, Loss: 0.0010
Epoch 2/20, Loss: 0.0016
Epoch 3/20, Loss: 0.0013
Epoch 4/20, Loss: 0.0007
Epoch 5/20, Loss: 0.0011
Epoch 6/20, Loss: 0.0011
Epoch 7/20, Loss: 0.0009
Epoch 8/20, Loss: 0.0013
Epoch 9/20, Loss: 0.0006
Epoch 10/20, Loss: 0.0012
Epoch 11/20, Loss: 0.0006
Epoch 12/20, Loss: 0.0007
Epoch 13/20, Loss: 0.0008
Epoch 14/20, Loss: 0.0010
Epoch 15/20, Loss: 0.0013
Epoch 16/20, Loss: 0.0007
Epoch 17/20, Loss: 0.0012
Epoch 18/20, Loss: 0.0007
Epoch 19/20, Loss: 0.0006
Epoch 20/20, Loss: 0.0010


In [10]:
best_validation_loss = float('inf')  # Initialize with a large value
num_epochs = 20
# Move model to the specified device

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    epoch_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

    # Validation
    model.eval()
    with torch.no_grad():
        validation_loss = 0.0
        correct = 0
        total = 0
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            validation_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        accuracy = 100 * correct / total
        avg_validation_loss = validation_loss / len(val_loader)
        #print(f"Validation Loss: {avg_validation_loss:.4f}, Accuracy: {accuracy:.2f}%")

        # Check if the current model has the best validation loss
        if avg_validation_loss < best_validation_loss:
            best_validation_loss = avg_validation_loss
            torch.save(model.state_dict(), "best_model.pt")


Epoch 1/20, Loss: 0.0012
Epoch 2/20, Loss: 0.0012
Epoch 3/20, Loss: 0.0009
Epoch 4/20, Loss: 0.0006
Epoch 5/20, Loss: 0.0007
Epoch 6/20, Loss: 0.0004
Epoch 7/20, Loss: 0.0005
Epoch 8/20, Loss: 0.0004
Epoch 9/20, Loss: 0.0004
Epoch 10/20, Loss: 0.0003
Epoch 11/20, Loss: 0.0006
Epoch 12/20, Loss: 0.0006
Epoch 13/20, Loss: 0.0003
Epoch 14/20, Loss: 0.0002
Epoch 15/20, Loss: 0.0003
Epoch 16/20, Loss: 0.0002
Epoch 17/20, Loss: 0.0002
Epoch 18/20, Loss: 0.0002
Epoch 19/20, Loss: 0.0002
Epoch 20/20, Loss: 0.0002


In [25]:
#Evaluating the model 
model_test_dataset = ImageFolder(root=data_dir + '/model_test', transform=transform)
test_loader = DataLoader(model_test_dataset, batch_size=batch_size, shuffle=False)

model.eval()

predictions = []
true_labels = []
example_inputs = None
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        example_input = images
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        predictions.extend(predicted.cpu().tolist())
        true_labels.extend(labels.cpu().tolist())

from sklearn.metrics import classification_report

print(classification_report(true_labels, predictions))


              precision    recall  f1-score   support

           0       0.86      0.91      0.88       106
           1       0.82      0.88      0.85        26
           2       1.00      1.00      1.00        41
           3       0.80      1.00      0.89       139
           4       0.99      0.72      0.83       108
           5       0.86      0.47      0.61        38
           6       0.93      0.99      0.96        79

    accuracy                           0.88       537
   macro avg       0.89      0.85      0.86       537
weighted avg       0.89      0.88      0.87       537



In [16]:
#Prepare example image
import torch
import torchvision.transforms as transforms
from PIL import Image

# Load and preprocess the image
image_path = "dtset\\train\\angular_leafspot\\angular_leafspot2.jpg"
image = Image.open(image_path).convert("RGB")

# Define the transformations to be applied to the image
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to the desired input size of your model
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

# Apply the transformations to the image
input_example = transform(image)

# Expand the dimensions of the input example to match the expected batch size
input_example = input_example.unsqueeze(0)